In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [20]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (kiss):
                    R_backbone has       1750 params (~ 50.6) %
                    t_backbone has       1711 params (~ 49.4) %
total params:                 3461
cam2cam estimation => adding 3461 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@14.05.2021-15:58:19
#[do_train]: epoch    0 has started!
#[cam2cam]: training batch iter 0 losses: GEO ~ 6.297, TRANS ~ 219.853, POSE ~ 93924.375, ROTO ~ 6.347, 3D ~ 653.556, SELF ~ 18.515, TOTAL ~ 746.131
#[cam2cam]: training batch iter 0 MPJPE: ~ 495.898 mm
#[cam2cam]: training batch iter 1 losses: GEO ~ 6.410, TRANS ~ 219.210, POSE ~ 91630.828, ROTO ~ 6.737, 3D ~ 647.252, SELF ~ 18.997, TOTAL ~ 742.237
#[cam2cam]: training batch iter 1 MPJPE: ~ 508.273 mm
#[cam2cam]: training batch iter 2 losses: GEO ~ 6.562, TRANS ~ 219.370, POSE ~ 105795.734, ROTO ~ 7.408, 3D ~ 649.084, SELF ~ 21.625, TOTAL ~ 757.211
#[cam2cam]: training batch iter 